In [1]:
import t4

In [9]:
# p = t4.Package().set_dir('/', '../data/images_cropped/').push(
#     'quilt/open_images',
#     's3://alpha-quilt-storage',
#     registry='s3://alpha-quilt-storage'
# )

In [12]:
!mkdir ../data/ 2>/dev/null
!mkdir ../data/images_cropped/ 2>/dev/null

In [11]:
t4.Package.install(
    'quilt/open_images', 
    registry='s3://alpha-quilt-storage', 
    dest='../data/images_cropped/'
)

Hamburger/
  10006714784_9337d5d0e1_o_2727.jpg
  10065094283_0db2b64b2d_o_529.jpg
  10102600246_6385283711_o_2195.jpg
  10123662565_4ab592b952_o_570.jpg
  10167235144_dc04e909e8_o_1998.jpg
  10220357564_526e3d72e8_o_1728.jpg
  10238023766_0e01a4bde6_o_1501.jpg
  10238023766_0e01a4bde6_o_1502.jpg
  10243597516_43c314d41c_o_441.jpg
  10263055086_3f270d5e26_o_1688.jpg
  102724558_563aae7e57_o_2142.jpg
  10273535936_7db00fb048_o_2039.jpg
  1036912912_4355f534ae_o_2695.jpg
  10382023383_5a64d7bb64_o_1522.jpg
  10386871386_195084b25b_o_378.jpg
  10386871386_195084b25b_o_379.jpg
  10395939425_a7470f4ab6_o_1612.jpg
Hot_dog/
Sandwich/

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    '../data/images_cropped/',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '../data/images_cropped/',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.losses import categorical_crossentropy


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss=categorical_crossentropy,
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
import pathlib

sample_size = len(list(pathlib.Path('../data/images_cropped/').rglob('./*')))
batch_size = 16

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=sample_size // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=round(sample_size * 0.2)
)

In [ ]:
model.to_csv("/opt/ml/model/clf.h5")